In [1]:
from vectorai import ViClient
import json
import xmltodict as xtd
import numpy as np
import os
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import PorterStemmer
from joblib import Parallel, delayed
from gensim import corpora
from gensim import models
import pandas as pd
from gensim import similarities
from operator import itemgetter

username = 'cesar-cabeza'
api_key = 'amdXU0VYVUJXN1RRYmVKWGUwcG46Ni1zRW15R01SNFNmLUZ4Ry00MHFmZw'

vi_client = ViClient(username=username, api_key = api_key)
from vectorai.models.deployed import ViText2Vec
text_encoder = ViText2Vec(username, api_key)


Logged in. Welcome cesar-cabeza. To view list of available collections, call list_collections() method.


In [7]:
vi_client.random_documents("corpus-test", page_size = 1, seed = 8)

{'documents': [{'_id': 'a998defbe9236dc5f7880910ccb4b2c6f350879d',
   'document': {'title': 'IFITM3 directly engages and shuttles incoming virus particles to lysosomes',
    'abstract': ['Interferon-induced transmembrane proteins (IFITMs) (1, 2 and 3) have emerged as important innate immune effectors that prevent diverse virus infections in vertebrates. However, the cellular mechanisms and live cell imaging of these small membrane proteins have been challenging to evaluate during viral entry of mammalian cells. Using CRISPR-Cas9-mediated IFITM-mutant cell lines, we demonstrate that human IFITM1, 2 and 3 act cooperatively and function in dosedependent fashion in interferon-stimulated cells. Through site-specific fluorophore tagging and live cell imaging studies, we show IFITM3 is on endocytic vesicles that fuse with incoming virus particles and enhances the trafficking of this pathogenic cargo to lysosomes. IFITM3 trafficking is specific to restricted viruses, requires S-palmitoylation 

In [70]:
vi_client.delete_collection('corpus-test')

{'status': 'complete', 'message': 'corpus-test deleted'}

In [71]:
path_test = "B:/document_parser/document_parses/test"

In [72]:
def load_corpus_single(path_corpus, client, username, api_key):
    text_encoder = ViText2Vec(username, api_key)
    directory = os.fsencode(path_corpus)  
    documents = []
    counter = 1
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        with open("\\".join([path_corpus, filename])) as file:
            file_json = json.load(file)
            file_json = preprocess_document(file_json)
        documents.append(file_json)
        if(counter % 139 == 0):
            client.insert_documents("corpus-test", documents, models = {"document": text_encoder.encode})            
            documents = []
        counter += 1
    return 

def preprocess_document(file):
    returned = {}
    returned["_id"]= file["paper_id"]
    title = file["metadata"]["title"]
    abstract = [paragraph["text"] for paragraph in file["abstract"]]
    text = [paragraph["text"] for paragraph in file["body_text"]]
    returned["document"] = {"title": title, "abstract":abstract, "text": text}
    return returned

In [73]:
load_corpus_single(path_test, vi_client, username, api_key)

In [74]:
vi_client.list_collections()

['corpus-test']

In [75]:
text_encoder = ViText2Vec(username, api_key)

In [89]:
results = vi_client.search('corpus-test', text_encoder.encode('papers abour '), 'document_vector_', page_size=10)

In [90]:
results

{'status': 'error',
 'message': 'Something went wrong, please raise the error with maintainer.',
 'error': 'Unknown Error'}

In [87]:
results["results"][0]["document"]["title"]

'PfSWIB, a potential chromatin regulator for var gene regulation and parasite development in Plasmodium falciparum'

In [88]:
[results["results"][i]["document"]["title"] for i in range(len(results["results"]))]

['PfSWIB, a potential chromatin regulator for var gene regulation and parasite development in Plasmodium falciparum',
 'Correlation between antimicrobial consumption and incidence of health-care- associated infections due to methicillin- resistant Staphylococcus aureus and vancomycin-resistant enterococci at a university hospital in Taiwan from 2000 to 2010',
 'Full Title: A systematic review of MERS-CoV (Middle East Respiratory Syndrome Coronavirus) 2 seroprevalence and viral RNA prevalence in dromedary camels: implications for animal vaccination',
 'Supplementary Information An eco-epidemiological study of Morbilli-related paramyxovirus infection in Madagascar bats reveals host-switching as the dominant macro-evolutionary mechanism',
 'FDA Perspectives on Diagnostic Device Clinical Studies for Respiratory Infections',
 'Innovations in research ethics governance in humanitarian settings',
 'Laboratory-based surveillance of hospital-acquired respiratory virus infection in a tertiary ca